In [28]:
import pandas as pd
import numpy as np
import networkx as nx
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('download', delim_whitespace=True, header=None)

In [3]:
with open('download') as f:
    foo = f.readline()

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8
0,-r--r--r--,1,ftp,ftp,2038334,Jul,30,2013,lookUpBlockIDToBlockNumNam.csv
1,-r--r--r--,1,ftp,ftp,548,Jul,30,2013,lookUpSignUpPeriods.csv
2,-r--r--r--,1,ftp,ftp,4700,Mar,14,2013,read_me.txt
3,-r--r--r--,1,ftp,ftp,13719074,Jul,30,2013,sfmtaAVL04042012_04112012.zip
4,-r--r--r--,1,ftp,ftp,59119742,Jan,2,2013,sfmtaAVLRawData01012013.csv


In [5]:
1.*df[4].sum()/(2**30)

76.71441589575261

In [6]:
2038334./(2**20)

1.9439067840576172

In [7]:
stops = pd.read_csv('data/google_transit/stops.txt')

In [8]:
stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4627 entries, 0 to 4626
Data columns (total 7 columns):
stop_id      4627 non-null int64
stop_name    4627 non-null object
stop_desc    4627 non-null object
stop_lat     4627 non-null float64
stop_lon     4627 non-null float64
zone_id      4627 non-null object
stop_url     4627 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 253.1+ KB


In [9]:
stops.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
0,98,2ND ST & MARKET ST,,37.789255,-122.401225,,
1,100,STEUART ST & FRANCISCO ST,,37.794463,-122.394607,,
2,202,COLUMBUS AVE & Pacific Ave,,37.797055,-122.405669,,
3,210,LINCOLN WAY & 19TH AVE,,37.765375,-122.477197,,
4,383,19TH AVE & LINCOLN WAY,,37.765375,-122.477197,,


In [10]:
routes = pd.read_csv('data/google_transit/routes.txt')

In [11]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,11047,SFMTA,1,CALIFORNIA,,3,,,
1,1033,SFMTA,1AX,CALIFORNIA A EXPRESS,,3,,,
2,1034,SFMTA,1BX,CALIFORNIA B EXPRESS,,3,,,
3,1031,SFMTA,31AX,BALBOA A EXPRESS,,3,,,
4,1032,SFMTA,31BX,BALBOA B EXPRESS,,3,,,


In [12]:
trips = pd.read_csv('data/google_transit/trips.txt')

In [13]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,11047,2,6858659,Geary + 33rd Avenue,0,115,135594
1,11047,1,7048008,Geary + 33rd Avenue,0,110,135595
2,11047,1,7048006,Geary + 33rd Avenue,0,116,135595
3,11047,1,7048005,Geary + 33rd Avenue,0,103,135595
4,11047,1,7048004,Geary + 33rd Avenue,0,113,135595


In [14]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29087 entries, 0 to 29086
Data columns (total 7 columns):
route_id         29087 non-null int64
service_id       29087 non-null int64
trip_id          29087 non-null int64
trip_headsign    29071 non-null object
direction_id     29087 non-null int64
block_id         29087 non-null int64
shape_id         29087 non-null int64
dtypes: int64(6), object(1)
memory usage: 1.6+ MB


In [15]:
len(trips['trip_id'].unique())

29087

In [16]:
stop_times = pd.read_csv('data/google_transit/stop_times.txt')

In [176]:
stop_times.head(30)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,6858631,26:00:00,26:00:00,4015,1,,,,
1,6858631,26:00:45,26:00:45,6294,2,,,,
2,6858631,26:01:33,26:01:33,6290,3,,,,
3,6858631,26:02:00,26:02:00,6314,4,,,,
4,6858631,26:02:35,26:02:35,6307,5,,,,
5,6858631,26:03:17,26:03:17,6302,6,,,,
6,6858631,26:03:51,26:03:51,6299,7,,,,
7,6858631,26:04:21,26:04:21,6316,8,,,,
8,6858631,26:05:00,26:05:00,6312,9,,,,
9,6858631,26:05:54,26:05:54,6315,10,,,,


In [18]:
shapes = pd.read_csv('data/google_transit/shapes.txt')

In [72]:
shapes.head(10)

,shape_id,shape_pt_lon,shape_pt_lat,shape_pt_sequence,shape_dist_traveled
0,135594,-122.446805,37.787266,1,0
1,135594,-122.448481,37.787054,2,149
2,135594,-122.450131,37.786842,3,296
3,135594,-122.450238,37.786822,4,306
4,135594,-122.451771,37.786624,5,443
5,135594,-122.453300,37.786433,6,579
6,135594,-122.453419,37.786422,7,590
7,135594,-122.454787,37.786248,8,712
8,135594,-122.455062,37.786213,9,737
9,135594,-122.455878,37.786109,10,810


In [21]:
for row in shapes.head().iterrows():
    print row

(0, shape_id               135594.000000
shape_pt_lon             -122.446805
shape_pt_lat               37.787266
shape_pt_sequence           1.000000
shape_dist_traveled         0.000000
Name: 0, dtype: float64)
(1, shape_id               135594.000000
shape_pt_lon             -122.448481
shape_pt_lat               37.787054
shape_pt_sequence           2.000000
shape_dist_traveled       149.000000
Name: 1, dtype: float64)
(2, shape_id               135594.000000
shape_pt_lon             -122.450131
shape_pt_lat               37.786842
shape_pt_sequence           3.000000
shape_dist_traveled       296.000000
Name: 2, dtype: float64)
(3, shape_id               135594.000000
shape_pt_lon             -122.450238
shape_pt_lat               37.786822
shape_pt_sequence           4.000000
shape_dist_traveled       306.000000
Name: 3, dtype: float64)
(4, shape_id               135594.000000
shape_pt_lon             -122.451771
shape_pt_lat               37.786624
shape_pt_sequence           5

In [23]:
response = requests.get('http://webservices.nextbus.com/service/publicXMLFeed?command=vehicleLocations&a=sf-muni')

In [26]:
response.text

u'<?xml version="1.0" encoding="utf-8" ?> \n<body copyright="All data copyright San Francisco Muni 2016.">\n<Error shouldRetry="false">\n  last time "t" parameter must be specified in query string\n</Error>\n<vehicle id="8217" routeTag="28" dirTag="28___O_F00" lat="37.80237" lon="-122.42482" secsSinceReport="28" predictable="true" heading="161" speedKmHr="7"/>\n<vehicle id="6247" routeTag="14R" dirTag="14R__O_F00" lat="37.71841" lon="-122.43962" secsSinceReport="13" predictable="true" heading="210" speedKmHr="50"/>\n<vehicle id="8179" routeTag="43" dirTag="43___O_F00" lat="37.80501" lon="-122.43302" secsSinceReport="69" predictable="true" heading="83" speedKmHr="0"/>\n<vehicle id="6508" routeTag="38R" dirTag="38R__O_F00" lat="37.78304" lon="-122.44257" secsSinceReport="5" predictable="true" heading="261" speedKmHr="38"/>\n<vehicle id="6221" routeTag="38R" dirTag="38R__I_F00" lat="37.7808599" lon="-122.46586" secsSinceReport="22" predictable="true" heading="87" speedKmHr="35"/>\n<vehicl

In [56]:
soup = BeautifulSoup(response.text, 'lxml')

In [57]:
len(soup.findAll('vehicle'))

550

In [58]:
soup.findAll('vehicle')[0]#['lat']

<vehicle dirtag="28___O_F00" heading="161" id="8217" lat="37.80237" lon="-122.42482" predictable="true" routetag="28" secssincereport="28" speedkmhr="7"></vehicle>

In [101]:
stop_tuples = stops.apply(lambda x: (x['stop_id'], {'stop_name':x['stop_name'],\
                                      'stop_lat':x['stop_lat'], 'stop_lon':x['stop_lon']}), axis=1)

In [117]:
stop_tuples[0]

(98,
 {'stop_lat': 37.789255,
  'stop_lon': -122.401225,
  'stop_name': '2ND ST & MARKET ST'})

In [205]:
G = nx.DiGraph()

In [206]:
G.add_nodes_from(stop_tuples)

In [207]:
G.node[98]

{'stop_lat': 37.789255,
 'stop_lon': -122.401225,
 'stop_name': '2ND ST & MARKET ST'}

In [208]:
for trip_id in trips['trip_id'].unique():
    trip_stops = stop_times[stop_times['trip_id'] == trip_id]
    for stop in trip_stops.iterrows():
        if stop[1]['stop_sequence'] == 1:
            last_stop = stop[1]['stop_id']
            continue
        if G.has_edge(last_stop, stop[1]['stop_id']):
            G.edge[last_stop][stop[1]['stop_id']]['trips'].append(\
                             {'trip_id': trip_id,\
                              'arr':stop[1]['arrival_time'],\
                              'route':trips[trips['trip_id']==trip_id]['route_id'].values[0]})            
        else:
            G.add_edge(last_stop, stop[1]['stop_id'],\
                   {'trips': [\
                             {'trip_id': trip_id,\
                              'arr':stop[1]['arrival_time'],\
                              'route':trips[trips['trip_id']==trip_id]['route_id'].values[0]}
                             ]})

        last_stop = stop[1]['stop_id']

In [210]:
G.number_of_edges()

4343

In [211]:
G.number_of_nodes()

4627

In [294]:
nx.shortest_path(G, 98, 4555)

[98, 3849, 7227, 7520, 7334, 5509, 4555]

In [293]:
G.neighbors(5944)

[7680,
 7681,
 5638,
 5639,
 5640,
 5641,
 4109,
 3089,
 7188,
 5657,
 5658,
 5671,
 5677,
 5678,
 5684,
 5685,
 5689,
 3137,
 7264,
 98,
 5731,
 4725,
 7810,
 7302,
 6290,
 4121,
 6596,
 6299,
 6302,
 6307,
 6308,
 6314,
 6316,
 6326,
 6327,
 6328,
 6332,
 6333,
 6334,
 6337,
 4301,
 4818,
 4819,
 4820,
 4821,
 4825,
 4826,
 3811,
 3815,
 4346,
 4347,
 3849,
 3851,
 3855,
 6928,
 3857,
 7444,
 3863,
 3864,
 3865,
 3866,
 3871,
 3872,
 3882,
 3883,
 5941,
 5942,
 3895,
 3899,
 3900,
 3894,
 5943,
 6994,
 6511,
 6522,
 6523,
 6524,
 7549,
 7550,
 6019,
 7831,
 7564,
 5528,
 4017,
 4018,
 4021,
 4025,
 4028,
 3012,
 3013,
 6606,
 6611,
 6125,
 6649]

In [291]:
G.has_edge(7810, 5944)

False

In [292]:
G.edge[5944][7810]

{'trips': [{'arr': '17:34:09', 'route': 1031, 'trip_id': 6860333},
  {'arr': '17:22:24', 'route': 1031, 'trip_id': 6860328},
  {'arr': '17:10:01', 'route': 1031, 'trip_id': 6860322},
  {'arr': '16:58:01', 'route': 1031, 'trip_id': 6860320},
  {'arr': '16:28:01', 'route': 1031, 'trip_id': 6860309},
  {'arr': '16:11:01', 'route': 1031, 'trip_id': 6860304},
  {'arr': '18:16:46', 'route': 1031, 'trip_id': 6860284},
  {'arr': '17:05:23', 'route': 1032, 'trip_id': 7013773},
  {'arr': '16:49:23', 'route': 1032, 'trip_id': 7013770},
  {'arr': '18:37:57', 'route': 1032, 'trip_id': 7013766},
  {'arr': '19:07:57', 'route': 1032, 'trip_id': 7008009},
  {'arr': '18:52:57', 'route': 1032, 'trip_id': 7007944},
  {'arr': '18:05:57', 'route': 1032, 'trip_id': 6860350},
  {'arr': '17:53:47', 'route': 1032, 'trip_id': 6860343},
  {'arr': '17:42:47', 'route': 1032, 'trip_id': 6860337},
  {'arr': '17:30:57', 'route': 1032, 'trip_id': 6860331},
  {'arr': '17:05:23', 'route': 1032, 'trip_id': 6860323},
  {'a

In [290]:
G.has_edge(3547,4555)

False

In [232]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2.)**2 + cos(lat1) * cos(lat2) * sin(dlon/2.)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [235]:
haversine(stops.loc[0]['stop_lon'], stops.loc[0]['stop_lat'], stops.loc[1]['stop_lon'], stops.loc[1]['stop_lat'])

0.8206931652358888

In [288]:
for row in stops.iterrows():
    stop = row[1]
    dists = stops.apply(lambda x: haversine(x['stop_lon'], x['stop_lat'], stop['stop_lon'], stop['stop_lat']),\
                       axis=1)
    dists = dists[dists < 0.5]
    dists = dists.drop(row[0])
    for close_stop in dists.iteritems():
        close_stop_id = stops.loc[close_stop[0]]['stop_id']
        if G.has_edge(stop['stop_id'], close_stop_id):
            G[stop['stop_id']][close_stop_id]['d'] = close_stop[1]
        else:
            G.add_edge(stop['stop_id'], close_stop_id , {'d':close_stop[1]})

In [289]:
G.number_of_edges()

186414

In [177]:
foo = stop_times[stop_times['trip_id'] == 7048004]

In [178]:
len(foo)

49

In [180]:
foo.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
976236,7048004,09:34:00,09:34:00,4015,1,,,,
976237,7048004,09:35:08,09:35:08,6294,2,,,,
976238,7048004,09:36:23,09:36:23,6290,3,,,,
976239,7048004,09:37:00,09:37:00,6314,4,,,,
976240,7048004,09:37:59,09:37:59,6307,5,,,,


In [175]:
trip_id = 6858633
for stop in foo.iterrows():
    if stop[1]['stop_sequence'] == 1:
        last_stop = stop[1]['stop_id']
        continue
    print (last_stop, stop[1]['stop_id'], {'trips': [\
                                                     {'trip_id': trip_id,\
                                                      'arr':stop[1]['arrival_time'],\
                                                      'route':trips[trips['trip_id']==trip_id]['route_id'].values[0]}
                                                     ]})
    last_stop = stop[1]['stop_id']

(4015, 6294, {'trips': [{'dep': '25:20:45', 'arr': '25:20:45', 'trip_id': 6858633, 'route': 11047}]})
(6294, 6290, {'trips': [{'dep': '25:21:33', 'arr': '25:21:33', 'trip_id': 6858633, 'route': 11047}]})
(6290, 6314, {'trips': [{'dep': '25:22:00', 'arr': '25:22:00', 'trip_id': 6858633, 'route': 11047}]})
(6314, 6307, {'trips': [{'dep': '25:22:35', 'arr': '25:22:35', 'trip_id': 6858633, 'route': 11047}]})
(6307, 6302, {'trips': [{'dep': '25:23:17', 'arr': '25:23:17', 'trip_id': 6858633, 'route': 11047}]})
(6302, 6299, {'trips': [{'dep': '25:23:51', 'arr': '25:23:51', 'trip_id': 6858633, 'route': 11047}]})
(6299, 6316, {'trips': [{'dep': '25:24:21', 'arr': '25:24:21', 'trip_id': 6858633, 'route': 11047}]})
(6316, 6312, {'trips': [{'dep': '25:25:00', 'arr': '25:25:00', 'trip_id': 6858633, 'route': 11047}]})
(6312, 6315, {'trips': [{'dep': '25:25:54', 'arr': '25:25:54', 'trip_id': 6858633, 'route': 11047}]})
(6315, 6301, {'trips': [{'dep': '25:26:39', 'arr': '25:26:39', 'trip_id': 6858633,

In [295]:
import cPickle as pickle

In [298]:
with open('graph.pkl','w') as f:
    pickle.dump(G, f)

In [317]:
g = nx.all_shortest_paths(G, 98, 4555)
for i in xrange(5):
    print g.next()

[98, 7680, 6524, 6695, 3104, 5510, 4555]
[98, 7681, 6524, 6695, 3104, 5510, 4555]
[98, 6019, 6524, 6695, 3104, 5510, 4555]
[98, 5640, 6524, 6695, 3104, 5510, 4555]
[98, 5642, 6524, 6695, 3104, 5510, 4555]
